In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# output for voacb control

In [21]:
import pandas as pd
import os

path = os.getcwd()
jasist_output_path = path+"\\output for vocab control\\jasist\\co-occurrence\\"
scient_output_path = path+"\\output for vocab control\\scientometrics\\co-occurrence\\"
files = os.listdir(scient_output_path)

In [22]:
files

['2010-2015_scient_cooccur_cosine_similarity_edge.csv',
 '2010-2016_scient_cooccur_cosine_similarity_edge.csv',
 '2010-2017_scient_cooccur_cosine_similarity_edge.csv',
 '2010-2018_scient_cooccur_cosine_similarity_edge.csv',
 '2010-2019_scient_cooccur_cosine_similarity_edge.csv',
 'scientometrics_cooccur_2010-15_lemma_nodes.xlsx',
 'scientometrics_cooccur_2010-16_lemma_nodes.xlsx',
 'scientometrics_cooccur_2010-17_lemma_nodes.xlsx',
 'scientometrics_cooccur_2010-18_lemma_nodes.xlsx',
 'scientometrics_cooccur_2010-19_lemma_nodes.xlsx']

In [23]:
cosine_similarity_edge_files = []
lemma_node_files = []
for f in files:
    if f.endswith('.csv'):
        cosine_similarity_edge_files.append(f)
    elif f.endswith('.xlsx'):
        lemma_node_files.append(f)

In [26]:
df_all_edges = pd.DataFrame()
for f in cosine_similarity_edge_files:
    df_time = pd.DataFrame(columns=['time'])
    df_iter = pd.read_csv(scient_output_path+f)
    df_iter = df_iter[['source', 'target', 'weight']]
    f = f.split('_')[0]
    df_time['time'] = ['{}'.format(f)]*len(df_iter)
    df_iter = df_time.join(df_iter)
    df_all_edges = df_all_edges.append(df_iter)

df_all_edges

,time,source,target,weight
0,2010-2015,1868,842,1.000000
1,2010-2015,1946,3758,1.000000
2,2010-2015,3758,1946,1.000000
3,2010-2015,4333,189,0.916667
4,2010-2015,1192,1743,0.916667
...,...,...,...,...
1715051,2010-2019,1328,2847,0.002684
1715052,2010-2019,644,2740,0.002615
1715053,2010-2019,922,256,0.002561
1715054,2010-2019,7455,6409,0.002516


In [27]:
df_all_edges = df_all_edges.sort_values(by=['weight', 'time'], ascending=[False, True])
# df_all_edges[df_all_edges['weight']>0.9].to_excel('temp_scient_cooccur_cosine_edge.xlsx')

In [30]:
df_all_nodes = pd.DataFrame()
for f in lemma_node_files:
    df_time = pd.DataFrame(columns=['time'])
    df_iter = pd.read_excel(scient_output_path+f, index_col=0)
    f = f.split('_')[2]
    df_time['time'] = ['{}'.format(f)]*len(df_iter)
    df_iter = df_time.join(df_iter)
    df_all_nodes = df_all_nodes.append(df_iter)
    
df_all_nodes

,time,id,label,LEMMA,STEM
0,2010-15,1,"""project 985"" university",project university,project univers
1,2010-15,2,'journal impact factor,journal impact factor,journal impact factor
2,2010-15,3,(h)over-bar,bar,bar
3,2010-15,4,(h)over-bar-core,bar core,bar core
4,2010-15,5,2-mode network,network,network
...,...,...,...,...,...
7575,2010-19,7576,zipf-mandelbrot-pareto law,zipf mandelbrot pareto law,zipf mandelbrot pareto law
7576,2010-19,7577,zone identification,zone identification,zone identif
7577,2010-19,7578,zoology,zoology,zoolog
7578,2010-19,7579,zoonotic disease,zoonotic disease,zoonot diseas


In [31]:
df_all_nodes = df_all_nodes.sort_values(by=['label', 'time'])
# df_all_nodes.to_excel('temp_scient_cooccur_lemma_nodes.xlsx')

In [32]:
import time

def replace_id_to_label(edge):
    new_edge = pd.DataFrame()
    label_arr = list(node['label'])
    # cost_time = []
    # start_time = time.time()
    for i in range(len(edge)):

    #     if i%10000==0:
    #         end_time = time.time()
    #         print('Elaspsed Time: {}, \t number of data: {}'.format(end_time-start_time, i))
    # #         cost_time.append(end_time-start_time)
    #         start_time = time.time()
        source_id = int(edge.at[i, 'source'])-1
        target_id = int(edge.at[i, 'target'])-1
        weight = edge.at[i, 'weight']
        source_label = label_arr[source_id]
        target_label = label_arr[target_id]

        new_edge.at[i, 'source'] = source_label
        new_edge.at[i, 'target'] = target_label
        new_edge.at[i, 'weight'] = weight
        
    return new_edge

In [35]:
t_node = sorted(list(df_all_nodes['time'].unique()))
t_edge = sorted(list(df_all_edges['time'].unique()))
new_edge_list = []
for i in range(len(t_node)):
    
    node = df_all_nodes[df_all_nodes['time']==t_node[i]]
    edge = df_all_edges[df_all_edges['time']==t_edge[i]]
    edge = edge[edge['weight']>=0.5]
    new_edge = replace_id_to_label(edge)
    new_edge.to_excel(os.getcwd()+"\\output for vocab control\\scientometrics\\co-occurrence\\{}_scient_label_edges_5.xlsx".format(t_node[i]))